<a href="https://colab.research.google.com/github/GENSHEHU/fine_tuning_nysc_byelaw/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing required libraries
!pip install tensorflow
!pip install transformers
!pip install PyPDF2
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [2]:
#importing required modules
from google.colab import drive
import os
import PyPDF2
import re
import json
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import tensorflow as tf
from transformers import Trainer, TrainingArguments
from transformers import pipeline

In [3]:
# Verifying and running code on GPU
print(tf.config.list_physical_devices('GPU'))
with tf.device('GPU:0'):
    tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    print(tensor)

[]
tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


In [5]:
# Mounting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Defining PDF path
pdf_path = '/content/nysc_bye_law.pdf'
json_path = '/content/nysc_bye_law.json'
model_save_path = '/content/drive/MyDrive/gpt2_finetuned'

#create the model save directory if it dosen't exist
os.makedirs(model_save_path, exist_ok=True)

In [7]:
# Function to extract text from the PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
      reader = PyPDF2.PdfReader(file)
      text = ''
      for page in reader.pages:
          text += page.extract_text() + '\n'
    return text

In [8]:
# Load GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#Set the padding token
tokenizer.pad_token = tokenizer.eos_token
model = TFGPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [9]:
# Function to clean and split texts into paragraphs
def clean_and_split_text(text):

# Replacing multiple space with single space
    text = re.sub(r'\s+', '', text)

# Split into paragraphs based on double new lines
    paragraphs = text.split('\n\n')

# Cleaning paragraphs and removing unwanted characters
    cleaned_paragraphs = [p.strip() for p in paragraphs if p.strip() and not p.isdigit() and len(p) > 10]
    return cleaned_paragraphs
    for p in paragraphs:
        p = p.strip()

# Filtering out page numbers, headers, short lines
        if p and not p.isdigit() and len(p) > 10:
            cleaned_paragraphs.append(p)
    return cleaned_paragraphs

# Extract raw text from the pdf
raw_text = extract_text_from_pdf(pdf_path)

# Cleaning and spliting into paragraphs
paragraphs = clean_and_split_text(raw_text)

# Printing example to verify extraction
print("Sample Paragraphs:")
for p in paragraphs[:3]:
    print(p[:100] + "...")

Sample Paragraphs:
NATIONALYOUTHSERVICECORPS(NYSC)BYE-LAWS(Revised2011)1.MOTTOOFTHENATIONALYOUTHSERVICECORPS•TheMottoof...


In [10]:
# Function to save data to JSON
def save_to_json(data, json_path):
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=2)

# Converting paragraphs into a list of dictionaries
dataset = [{"text": p} for p in paragraphs]

# Saving dataset to JSON
save_to_json(dataset, json_path)

# Verify by loading and printing a sample
with open(json_path, 'r') as f:
    loaded_dataset = json.load(f)
print("Sample from JSON:")
print(loaded_dataset[0]['text'][:100] + "...")

Sample from JSON:
NATIONALYOUTHSERVICECORPS(NYSC)BYE-LAWS(Revised2011)1.MOTTOOFTHENATIONALYOUTHSERVICECORPS•TheMottoof...


In [11]:
# Function to tokenize text
def tokenize_function(examples):
    return tokenizer(examples['text'],
truncation=True, padding='max_length',
max_length=1024)

# Tokenzing the dataset
tokenized_dataset = [tokenizer(p['text'], truncation=True, padding='max_length', max_length=1024) for p in dataset]

# Generator function for TensorFlow dataset
def generator():
    for item in tokenized_dataset:
        yield {'input_ids': item['input_ids'], 'attention_mask': item['attention_mask']}

# Print dataset size for debugging
print(f"Dataset size: {len(dataset)} paragraphs")
for i, p in enumerate(dataset[:3]):  # Sample first 3 entries
    print(f"Sample {i+1}: {p['text'][:100]}...")

# Create TensorFlow dataset with batching
tf_dataset = tf.data.Dataset.from_generator(
    generator,
    output_types={'input_ids': tf.int32, 'attention_mask': tf.int32},
    output_shapes={'input_ids': [1024], 'attention_mask': [1024]}
).batch(1)

print("Dataset ready for training!")

Dataset size: 1 paragraphs
Sample 1: NATIONALYOUTHSERVICECORPS(NYSC)BYE-LAWS(Revised2011)1.MOTTOOFTHENATIONALYOUTHSERVICECORPS•TheMottoof...
Dataset ready for training!


In [12]:
#Checking file type
for i, example in enumerate(dataset[:3]):
    print(f"Entry {i}: Type={type(example['text'])}, Content={example['text'][:50]}...")

Entry 0: Type=<class 'str'>, Content=NATIONALYOUTHSERVICECORPS(NYSC)BYE-LAWS(Revised201...


In [13]:
#Converting entryies to strings
paragraphs = [str(p) for p in paragraphs if p is not None]
dataset = [{"text": p} for p in paragraphs]
#Removing empty paragraphs
paragraphs = [p for p in paragraphs if p.strip()]
dataset = [{"text": p} for p in paragraphs]


In [14]:
#using ftfy to fix encoding issues
!pip install ftfy
import ftfy
paragraphs = [ftfy.fix_text(p) for p in paragraphs]
dataset = [{"text": p} for p in paragraphs]

paragraphs = [p.encode('ascii', 'ignore').decode('ascii') for p in paragraphs]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00


In [15]:
#Spliting paragraphs into smaller chunks
def split_long_paragraphs(paragraphs, max_words=500):
    new_paragraphs = []
    for p in paragraphs:
        words = p.split()
        if len(words) > max_words:
            for i in range(0, len(words), max_words):
                chunk = ' '.join(words[i:i + max_words])
                new_paragraphs.append(chunk)
        else:
            new_paragraphs.append(p)
    return new_paragraphs

paragraphs = split_long_paragraphs(paragraphs)
dataset = [{"text": p} for p in paragraphs]

In [16]:
#Tokenizing one piece
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token
test_text = dataset[0]['text']  # Assumes dataset is defined
tokenized_output = tokenizer(test_text, truncation=True, padding='max_length', max_length=1024)
print("Tokenized Output:", tokenized_output)

Tokenized Output: {'input_ids': [45, 29912, 36981, 4221, 35009, 53, 2149, 2943, 1581, 3705, 7, 12805, 6173, 8, 17513, 36, 12, 43, 12298, 50, 7, 18009, 1417, 9804, 8, 16, 13, 44, 29089, 6684, 37, 4221, 1677, 29912, 36981, 4221, 35009, 53, 2149, 2943, 1581, 3705, 464, 44, 1252, 2238, 701, 258, 16186, 56, 1536, 16177, 10606, 862, 7, 1456, 1437, 637, 260, 18186, 1462, 459, 258, 1, 16177, 10606, 862, 4943, 49271, 1350, 1, 16177, 392, 32661, 879, 17, 13, 10970, 6489, 1961, 8264, 6109, 19522, 11205, 258, 16177, 10606, 862, 7, 1456, 1437, 637, 260, 18186, 1462, 292, 1, 19522, 4943, 1477, 5691, 549, 40075, 27206, 1746, 570, 330, 404, 8226, 701, 258, 16186, 10137, 469, 392, 12805, 6173, 10137, 469, 19052, 1169, 23675, 26141, 2004, 870, 10462, 6648, 12, 259, 34, 567, 9926, 1078, 258, 13989, 341, 16544, 13, 2025, 4948, 1491, 1929, 1659, 1768, 273, 5420, 385, 395, 418, 549, 40075, 27206, 1746, 16891, 258, 16186, 392, 1169, 12805, 6173, 10137, 70, 408, 18323, 11181, 2228, 1525, 1169, 21111, 36699, 3

In [17]:
tokenized_dataset = [tokenizer(p['text'],
truncation=True, padding='max_length',
max_length=1024) for p in dataset]

In [37]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Define loss function (GPT-2 uses causal LM loss)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

# Training parameters
epochs = 30
steps_per_epoch = max(1, len(tokenized_dataset) // 2)

In [ ]:
# Custom training step
@tf.function
def train_step(inputs):
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    with tf.GradientTape() as tape:
        outputs = model(input_ids, attention_mask=attention_mask, training=True)
        logits = outputs.logits
        shift_logits = logits[:, :-1, :]
        shift_labels = input_ids[:, 1:]
        shift_mask = tf.cast(attention_mask[:, 1:], tf.float32)
        loss = loss_fn(shift_labels, shift_logits)
        masked_loss = loss * shift_mask
        loss_value = tf.reduce_sum(masked_loss) / tf.reduce_sum(shift_mask)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    total_loss = 0
    step_count = 0
    for step, batch in enumerate(tf_dataset):
        loss_value = train_step(batch)
        total_loss += loss_value
        step_count += 1
        if step % 10 == 0:  # Print every 10 steps
            print(f"Step {step}/{steps_per_epoch}, Loss: {loss_value:.4f}")
    avg_loss = total_loss / step_count if step_count > 0 else float('inf')
    print(f"Average Loss for Epoch {epoch + 1}: {avg_loss:.4f}")

# Save the fine-tuned model and tokenizer
model_save_path = "/content/drive/MyDrive/gpt2_finetuned"
model.save_pretrained("./model_output")
tokenizer.save_pretrained("./model_output")

print("Training completed and model saved!")

Epoch 1/30


In [20]:
"""
# Define training parameters
training_args = TrainingArguments(
    output_dir="./model_output",    # model save directory
    overwrite_output_dir=True,      # overwrite output directory if it exists
    num_train_epochs=3,             # number of training epochs
    per_device_train_batch_size=4,  # batch size per device
    save_steps=500,                 # save checkpoint every 500 steps
    save_total_limit=2,             # keep the last 2 checkpoint only
    logging_dir="./logs",           # log directory
)

# Initializing the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tf_dataset,
)

# Starting training
trainer.train()
"""

'\n# Define training parameters\ntraining_args = TrainingArguments(\n    output_dir="./model_output",    # model save directory\n    overwrite_output_dir=True,      # overwrite output directory if it exists\n    num_train_epochs=3,             # number of training epochs\n    per_device_train_batch_size=4,  # batch size per device\n    save_steps=500,                 # save checkpoint every 500 steps\n    save_total_limit=2,             # keep the last 2 checkpoint only\n    logging_dir="./logs",           # log directory\n)\n\n# Initializing the trainer\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    train_dataset=tf_dataset,\n)\n\n# Starting training\ntrainer.train()\n'

In [21]:
# Saving the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/gpt2_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2_finetuned/vocab.json',
 '/content/drive/MyDrive/gpt2_finetuned/merges.txt',
 '/content/drive/MyDrive/gpt2_finetuned/added_tokens.json')

In [22]:
# Loading the trained model for text generation
generator = pipeline('text-generation',
model=model_save_path, tokenizer=tokenizer)

# Sample prompt related to NYSC Bye-Laws
prompt = "Section 1: All Corps Members shall"

# Generate text with adjustable parameters
output = generator(
    prompt,
    max_length=100,       # Maximum length for generated texts
    temperature=0.7,      # Randomness control
    top_k=50,             # Sample limiting to top 50 tokens
    top_p=0.9,            # Nucleus sampling
    do_sample=True        # Enables sampling for varied outputs
)

# Printing generated texts
print("Generated Text:")
print(output[0]['generated_text'])

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/gpt2_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Device set to use 0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Text:
Section 1: All Corps Members shall, without delay, submit to the Chief of the Corps of Engineers and the Chief of the State and local governments of the State or of the Territory, a copy of any of the following: (a) The name, address, and telephone number of the person who, by virtue of his or her official duties, is to be authorized to act as the Secretary of the Interior. (b) The date, time, place, and manner of his or her attendance


In [23]:
# Sample prompt related to NYSC Bye-Laws
prompt = "Corps Members must"

# Generate text with adjustable parameters
output = generator(
    prompt,
    max_length=100,       # Maximum length for generated texts
    temperature=0.5,      # Randomness control
    top_k=40,             # Sample limiting to top 50 tokens
    top_p=0.85,            # Nucleus sampling
    do_sample=True        # Enables sampling for varied outputs
)

# Printing generated texts
print("Generated Text:")
print(output[0]['generated_text'])

Generated Text:
Corps Members must be registered with the Department of Justice.

Criminal Justice

Criminal Justice is a civil rights organization that works to improve the lives of black and brown people.

The organization's mission is to promote justice and to build a better society.

Criminal Justice is committed to building a better society by:

Promoting the rule of law and the rule of law of the United States.

Promoting civil rights, including the right to fair


In [24]:
# Load the trained model and tokenizer from the saved path
model_save_path = "/content/drive/MyDrive/gpt2_finetuned"  # Confirm this matches your save location
generator = pipeline('text-generation', model=model_save_path, tokenizer=tokenizer)

# Sample prompt related to NYSC Bye-Laws
prompt = "Section 1: All Corps Members shall"

# Generate text with refined parameters
output = generator(
    prompt,
    max_length=100,         # Maximum length for generated text
    truncation=True,        # Explicitly enable truncation to avoid warning
    temperature=0.6,        # Slightly lower for less randomness
    top_k=40,              # Focus on top 40 tokens
    top_p=0.85,            # Stricter nucleus sampling
    repetition_penalty=1.2, # Penalize repeated phrases
    do_sample=True,         # Enable sampling
    num_return_sequences=1  # Generate one sequence
)

# Print generated text
print("Generated Text:")
print(output[0]['generated_text'])

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/gpt2_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Device set to use 0


Generated Text:
Section 1: All Corps Members shall be entitled to receive an annual salary of $2,500.
(Source)


In [25]:
generator = pipeline('text-generation', model="/content/drive/MyDrive/gpt2_finetuned", tokenizer=tokenizer)
prompt = "Section 1: All Corps Members shall"
output = generator(prompt, max_length=100, truncation=True, temperature=0.6, top_k=40, top_p=0.85, repetition_penalty=1.2, do_sample=True)
print("Generated Text:")
print(output[0]['generated_text'])

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/gpt2_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Device set to use 0


Generated Text:
Section 1: All Corps Members shall be required to obtain a written certification from the Secretary of Defense, as set forth in subsection (b)(1)(B), that they have met all requirements for membership.
(2) If an individual is disqualified under paragraph 5 and has been convicted or pleaded guilty before any court on charges relating thereto pursuant"to section 3304,"the person may appeal against such conviction by filing with his district attorney within 60 days after he was found not eligible."The Department's


In [36]:
# Load your fine-tuned model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = TFGPT2LMHeadModel.from_pretrained(model_name)

# --- Helper: Load your training sequences ---
# For constrained generation, you may want to load the exact sequences from your dataset
# Here we assume you have a text file (or list) with one training sequence per line.
with open("training_sequences.txt", "r", encoding="utf-8") as f:
    training_sequences = {line.strip() for line in f if line.strip()}

# --- Generation function with deterministic decoding ---
def generate_text(prompt, max_length=100):
    # Encode prompt text
    input_ids = tokenizer.encode(prompt, return_tensors='tf')
    attention_mask = tf.ones_like(input_ids)

    # Generate text with deterministic settings:
    # Temperature 0 forces the model to choose the highest probability token.
    # top_k=1 and top_p=1 restrict the sampling to the most likely token.
    # do_sample=False disables random sampling.
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.0,   # Force deterministic output
        top_k=1,           # Only consider the top token
        top_p=1.0,         # No nucleus sampling
        do_sample=False    # Greedy decoding
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # --- Post-processing: Ensure exact match with training data ---
    # If your goal is to only output sequences present in your training data,
    # compare the generated text to your training sequences.
    if generated_text in training_sequences:
        return generated_text
    else:
        # Option 1: Return an error or empty string if no exact match is found.
        # Option 2: You could loop or try alternative decoding strategies.
        print("Generated sequence does not match any training example exactly.")
        return ""

# --- Example usage ---
prompt = "Every member shall attend regulary,"
result = generate_text(prompt, max_length=200)
print("Final Output:\n", result)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Generated sequence does not match any training example exactly.
Final Output:
 
